In [1]:
!pip install wandb pyyaml opencv-python;

In [2]:
import os
import yaml
import glob
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
import wandb
from wandb.integration.keras import WandbCallback

2025-05-03 15:49:10.014893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-03 15:49:10.032780: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-03 15:49:10.038152: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-03 15:49:10.051626: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-03 15:49:10.825113: W tensorflow/compiler/tf2

In [3]:
wandb.login(key="017feefe0af6702cda76aab121ec71cf3a362fec")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc
wandb: Currently logged in as: samutheman (rueedi-tobias-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [15]:
dataset_yaml = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml"
images_trainset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/train"
lables_trainset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/lables/train"
images_valset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/val"
lables_valset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/lables/val"
images_testset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/test"
lables_testset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/lables/test"


In [16]:
with open(dataset_yaml, "r") as f:
    config = yaml.safe_load(f)


In [17]:
# --- Parameters ---
IMG_SIZE = 128
GRID_SIZE = 4
CELLS = GRID_SIZE * GRID_SIZE
NUM_BOXES = 1  # one box per grid cell
BBOX_ATTRS = 4  # x, y, w, h, obj_confidence
OUTPUT_SHAPE = (GRID_SIZE, GRID_SIZE, NUM_BOXES * (BBOX_ATTRS + 1))

In [18]:
def build_yolo_model(input_shape=(128, 128, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(2)(x)  # 64x64
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)  # 32x32
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)  # 16x16
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)  # 8x8
    x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)  # 4x4
    x = layers.Conv2D(OUTPUT_SHAPE[-1], 1, activation='sigmoid')(x)  # final layer
    x = layers.Reshape(OUTPUT_SHAPE)(x)  # shape: (4, 4, B*(5+C))
    return tf.keras.Model(inputs, x)


In [19]:
def yolo_loss(y_true, y_pred):
    # Mean squared error over box + class + objectness
    loss = tf.reduce_mean(tf.square(y_true - y_pred))
    return loss

In [30]:
def load_yolo_dataset(image_dir, label_dir, img_size=IMG_SIZE):
    image_paths = sorted(glob.glob(os.path.join(image_dir, "*.[jJpP][pPnN][gG]")))
    x_data, y_data = [], []
    print(image_paths)

    for img_path in image_paths:
        filename = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(label_dir, f"{filename}.txt")

        # Load image
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size, img_size))
        img = img / 255.0
        x_data.append(img)

        # Create empty label grid
        label = np.zeros(OUTPUT_SHAPE, dtype=np.float32)

        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                lines = f.readlines()
            for line in lines:
                class_id, x, y, w, h = map(float, line.strip().split())
                grid_x = int(x * GRID_SIZE)
                grid_y = int(y * GRID_SIZE)
                if grid_x >= GRID_SIZE or grid_y >= GRID_SIZE:
                    continue

                # relative x,y,w,h within the cell
                cell_x = x * GRID_SIZE - grid_x
                cell_y = y * GRID_SIZE - grid_y
                label[grid_y, grid_x, :5] = [cell_x, cell_y, w, h, 1.0]
                label[grid_y, grid_x, 5 + int(class_id)] = 1.0  # class one-hot

        y_data.append(label)

    return np.array(x_data, dtype=np.float32), np.array(y_data, dtype=np.float32)


In [31]:
x_train, y_train = load_yolo_dataset(lables_trainset, images_trainset)
x_val, y_val     = load_yolo_dataset(lables_valset, images_valset)
if test_img_dir:
    x_test, y_test = load_yolo_dataset(lables_testset , images_testset)

[]
[]
[]


In [23]:
wandb.init(project="Own CNN", name="Detector", config=config)

In [25]:
print("x_train type:", type(x_train))
print("x_train shape:", getattr(x_train, 'shape', 'No shape'))
print("First element type:", type(x_train[0]))

x_train type: <class 'numpy.ndarray'>
x_train shape: (0,)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [12]:
model = build_yolo_model()
model.compile(optimizer='adam', loss=yolo_loss)

model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=30,
    batch_size=16,
    callbacks=[WandbCallback()]
)
if test_img_dir:
    loss = model.evaluate(x_test, y_test)
    print("Test Loss:", loss)

2025-05-03 15:49:15.557479: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13294 MB memory:  -> device: 0, name: NVIDIA A16, pci bus id: 0000:86:00.0, compute capability: 8.6
wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/30


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data:0", shape=(16,), dtype=float32). Expected shape (None, 128, 128, 3), but input has incompatible shape (16,)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(16,), dtype=float32)
  • training=True
  • mask=None